In [3]:
from google.colab import files
uploaded = files.upload()
import pandas as pd



Saving Tweets.csv to Tweets.csv


In [20]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.preprocessing import LabelEncoder

In [11]:
df = pd.read_csv('Tweets.csv')
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
df.isna().sum()

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5462
negativereason_confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_location                   4733
user_timezone                    4820
dtype: int64

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder()
# sentiment_transformed = ohe.fit_transform(df[['airline_sentiment']])

In [ ]:
# sentiment_transformed.toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [ ]:
# sentiment_df = pd.DataFrame(sentiment_transformed.toarray(), columns=ohe.get_feature_names_out(['airline_sentiment']),index=df.index)
# sentiment_df

,airline_sentiment_negative,airline_sentiment_neutral,airline_sentiment_positive
0,0.0,1.0,0.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
14635,0.0,0.0,1.0
14636,1.0,0.0,0.0
14637,0.0,1.0,0.0
14638,1.0,0.0,0.0


In [ ]:
# df = pd.concat([df.drop(columns = ['airline_sentiment']),sentiment_df],axis=1)
# df.head()

,tweet_id,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,airline_sentiment_negative,airline_sentiment_neutral,airline_sentiment_positive
0,570306133677760513,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),0.0,1.0,0.0
1,570301130888122368,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),0.0,0.0,1.0
2,570301083672813571,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),0.0,1.0,0.0
3,570301031407624196,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),1.0,0.0,0.0
4,570300817074462722,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),1.0,0.0,0.0


In [5]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
lable_encoder_gender = LabelEncoder()
df['airline_sentiment'] = lable_encoder_gender.fit_transform(df['airline_sentiment'])
df['airline_sentiment'].unique()

array([1, 2, 0])

In [6]:
X = df['text']
y=df['airline_sentiment']
X,y

(0                      @VirginAmerica What @dhepburn said.
 1        @VirginAmerica plus you've added commercials t...
 2        @VirginAmerica I didn't today... Must mean I n...
 3        @VirginAmerica it's really aggressive to blast...
 4        @VirginAmerica and it's a really big bad thing...
                                ...                        
 14635    @AmericanAir thank you we got on a different f...
 14636    @AmericanAir leaving over 20 minutes Late Flig...
 14637    @AmericanAir Please bring American Airlines to...
 14638    @AmericanAir you have my money, you change my ...
 14639    @AmericanAir we have 8 ppl so we need 2 know h...
 Name: text, Length: 14640, dtype: object,
 0        1
 1        2
 2        1
 3        0
 4        0
         ..
 14635    2
 14636    0
 14637    1
 14638    0
 14639    1
 Name: airline_sentiment, Length: 14640, dtype: int64)

In [7]:
X = X.str.lower()
X

,text
0,@virginamerica what @dhepburn said.
1,@virginamerica plus you've added commercials t...
2,@virginamerica i didn't today... must mean i n...
3,@virginamerica it's really aggressive to blast...
4,@virginamerica and it's a really big bad thing...
...,...
14635,@americanair thank you we got on a different f...
14636,@americanair leaving over 20 minutes late flig...
14637,@americanair please bring american airlines to...
14638,"@americanair you have my money, you change my ..."


In [9]:
# removing sepecial character
import nltk
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
print("started")
X=X.apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
print("Started")
X=X.apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url
print("Started")
X=X.apply(lambda x: re.sub(r'@\w+', '', str(x)))
## Remove html tags
print("Started")
X=X.apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
## Remove any additional spaces
print("Started")
X=X.apply(lambda x: " ".join(x.split()))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


started
Started
Started
Started
Started


In [10]:
X

,text
0,virginamerica dhepburn said
1,virginamerica plus youve added commercials exp...
2,virginamerica didnt today must mean need take ...
3,virginamerica really aggressive blast obnoxiou...
4,virginamerica really big bad thing
...,...
14635,americanair thank got different flight chicago
14636,americanair leaving 20 minutes late flight war...
14637,americanair please bring american airlines bla...
14638,americanair money change flight dont answer ph...


In [16]:
texts = df['text']
from sklearn.preprocessing import StandardScaler,LabelEncoder
lable_encoder_gender = LabelEncoder()
sentiments = lable_encoder_gender.fit_transform(df['airline_sentiment'])



In [17]:
def clean_text(text):
    text = re.sub(r'@\w+', '', text)     # remove handles
    text = re.sub(r'http\S+', '', text)  # remove URLs
    text = text.lower()
    return text.strip()

texts = texts.apply(clean_text)

In [19]:
sentiments

array([1, 2, 1, ..., 1, 0, 1])

In [21]:
tokenizer = Tokenizer(num_words=10000, oov_token="<UNK>")
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=50)


In [22]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=50),
    SimpleRNN(64, activation='tanh'),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')   # 3 classes
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.fit(padded, y, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20
366/366 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9915 - loss: 0.0302 - val_accuracy: 0.7804 - val_loss: 1.1497
Epoch 2/20
366/366 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9926 - loss: 0.0243 - val_accuracy: 0.7394 - val_loss: 1.2589
Epoch 3/20
366/366 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9880 - loss: 0.0363 - val_accuracy: 0.7671 - val_loss: 1.2393
Epoch 4/20
366/366 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9908 - loss: 0.0281 - val_accuracy: 0.7145 - val_loss: 1.3453
Epoch 5/20
366/366 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9918 - loss: 0.0246 - val_accuracy: 0.7514 - val_loss: 1.3324
Epoch 6/20
366/366 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9926 - loss: 0.0231 - val_accuracy: 0.7258 - val_loss: 1.4195
Epoch 7/20
366/366 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9927 - loss: 0.0233 - val_accuracy: 0.7121 - val_loss: 1.5858
Epoch 8/20
366/366 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9878 - loss: 0.0339 - val_accuracy: 0.

In [25]:
model.save('twitter_airline.h5')

In [27]:
import pickle
with open ('tokenizer.pkl','wb') as files:
  pickle.dump(tokenizer,files,protocol=pickle.HIGHEST_PROTOCOL)